In [4]:
from google.colab import files

uploaded = files.upload()

Saving rt-polarity.neg to rt-polarity.neg
Saving rt-polarity.pos to rt-polarity.pos


In [0]:
import numpy as np
import re
import tensorflow as tf
import datetime

In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "rt-polarity.neg" with length 612290 bytes
User uploaded file "rt-polarity.pos" with length 626168 bytes


In [0]:
import numpy as np
import re


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r", encoding='windows-1252').readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r", encoding='windows-1252').readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [0]:
from tensorflow.contrib import learn

def preprocess():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = load_data_and_labels("rt-polarity.pos", "rt-polarity.neg")

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))
    print(x)

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_percentage = .1 #Percentage of the training data to use for validation
    dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

In [0]:
class TextCNN(object):
  """
  CNN for text classification
  Uses an embedding layer, followed by a conv, max-pool and softmax layer
  """
  def __init__(self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters):
    self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
    self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
    
    W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
    self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
    #Because conv2d expects 4-dim tensor with channel dimension
    self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
    
    #Convolution and Max-Pooling Layers
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
      with tf.name_scope("conv-maxpool-%s" % filter_size):
        #Conv Layer
        filter_shape = [filter_size, embedding_size, 1, num_filters]
        W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
        conv = tf.nn.conv2d(
            self.embedded_chars_expanded,
            W,
            strides = [1,1,1,1],
            padding = "VALID",
            name = "conv"
        )
        # Apply nonlinearity
        h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
        # Max-pooling over the outputs
        pooled = tf.nn.max_pool(
            h,
            ksize = [1, sequence_length - filter_size + 1, 1, 1],
            strides = [1,1,1,1],
            padding = "VALID",
            name = "pool"
        )
        pooled_outputs.append(pooled)
    #Combine all the pooled features
    print(pooled_outputs)
    num_filters_total = num_filters * len(filter_sizes)
    self.h_pool = tf.concat(pooled_outputs, 3)
    self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
    # Add dropout
    with tf.name_scope("dropout"):
      self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
    
    with tf.name_scope("output"):
      W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
      b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
      self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
      self.predictions = tf.argmax(self.scores, 1, name="predictions")
      
    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
      losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)      
      self.loss = tf.reduce_mean(losses)
      
    # Calculate Accuracy
    with tf.name_scope("accuracy"):
      correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
      self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    

In [0]:
#FLAGS
embedding_dim = 128
filter_sizes = "3,4,5"
num_filters = 128
dropout_keep_prob = 0.5
batch_size = 64
num_epochs = 200
evaluate_every = 100 

In [0]:
def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================
    
    with tf.Graph().as_default():
        sess = tf.Session()    
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=embedding_dim,
                filter_sizes=list(map(int, filter_sizes.split(","))),
                num_filters=num_filters)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)


            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: dropout_keep_prob
                }
                _, step, loss, accuracy = sess.run(
                    [train_op, global_step, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, loss, accuracy = sess.run(
                    [global_step, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

            # Generate batches
            batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)              

In [5]:
x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()

Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
[[    1     2     3 ...     0     0     0]
 [    1    31    32 ...     0     0     0]
 [   57    58    59 ...     0     0     0]
 ...
 [   75    84  1949 ...     0     0     0]
 [    1  2191  2690 ...     0     0     0]
 [11512     3   147 ...     0     0     0]]
Vocabulary Size: 18758
Train/Dev split: 9596/1066


In [0]:
train(x_train, y_train, vocab_processor, x_dev, y_dev)

[<tf.Tensor 'conv-maxpool-3/pool:0' shape=(?, 1, 1, 128) dtype=float32>, <tf.Tensor 'conv-maxpool-4/pool:0' shape=(?, 1, 1, 128) dtype=float32>, <tf.Tensor 'conv-maxpool-5/pool:0' shape=(?, 1, 1, 128) dtype=float32>]
2018-12-12T20:37:25.860520: step 1, loss 5.8025, acc 0.46875
2018-12-12T20:37:25.970628: step 2, loss 3.45029, acc 0.578125
2018-12-12T20:37:26.085276: step 3, loss 3.49701, acc 0.515625
2018-12-12T20:37:26.191096: step 4, loss 2.57076, acc 0.5625
2018-12-12T20:37:26.300766: step 5, loss 2.96863, acc 0.5
2018-12-12T20:37:26.402603: step 6, loss 2.85766, acc 0.46875
2018-12-12T20:37:26.509803: step 7, loss 3.26028, acc 0.515625
2018-12-12T20:37:26.614847: step 8, loss 2.2191, acc 0.59375
2018-12-12T20:37:26.723845: step 9, loss 2.81255, acc 0.484375
2018-12-12T20:37:26.830305: step 10, loss 2.25934, acc 0.46875
2018-12-12T20:37:26.937084: step 11, loss 2.40631, acc 0.484375
2018-12-12T20:37:27.041042: step 12, loss 2.38637, acc 0.46875
2018-12-12T20:37:27.150925: step 13, l